In [ ]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib.parse
import time
import numpy as np
import pandas as pd
import logging
import sqlite3
from datetime import datetime

# ロギング設定
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='scraping.log'
)

# ヘッダー設定
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# SUUMOを東京都品川区大井町のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13109&oz=13109002&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50={}'

# データベース初期化関数
def init_database():
    conn = sqlite3.connect('suumo_properties.db')
    cursor = conn.cursor()
    
    # テーブル作成 (存在しない場合)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS properties (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        scrape_date TEXT,
        category TEXT,
        building_name TEXT,
        address TEXT,
        nearest_station1 TEXT,
        nearest_station2 TEXT,
        building_age TEXT,
        total_floors TEXT,
        room_floor TEXT,
        rent TEXT,
        management_fee TEXT,
        deposit TEXT,
        gratuity TEXT,
        layout TEXT,
        area TEXT,
        url TEXT,
        page_number INTEGER
    )
    ''')
    
    conn.commit()
    return conn, cursor

# データベースへ挿入関数
def insert_to_database(conn, cursor, data_samples, current_page):
    scrape_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # データベースへの一括挿入
    insert_query = '''
    INSERT INTO properties (
        scrape_date, category, building_name, address, 
        nearest_station1, nearest_station2, building_age, 
        total_floors, room_floor, rent, management_fee, 
        deposit, gratuity, layout, area, url, page_number
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    
    # データの準備
    insert_data = []
    for sample in data_samples:
        insert_data.append((
            scrape_date, 
            sample[0], sample[1], sample[2], 
            sample[3], sample[4], sample[5], 
            sample[6], sample[7], sample[8], 
            sample[9], sample[10], sample[11], 
            sample[12], sample[13], sample[14], 
            current_page
        ))
    
    # 一括挿入
    cursor.executemany(insert_query, insert_data)
    conn.commit()

    # プリント出力
    for sample in data_samples:
        print("---物件情報---")
        print(f"カテゴリ: {sample[0]}")
        print(f"建物名: {sample[1]}")
        print(f"住所: {sample[2]}")
        print(f"最寄り駅1: {sample[3]}")
        print(f"最寄り駅2: {sample[4]}")
        print(f"築年数: {sample[5]}")
        print(f"総階数: {sample[6]}")
        print(f"部屋階: {sample[7]}")
        print(f"家賃: {sample[8]}")
        print(f"管理費: {sample[9]}")
        print(f"敷金: {sample[10]}")
        print(f"礼金: {sample[11]}")
        print(f"間取り: {sample[12]}")
        print(f"面積: {sample[13]}")
        print(f"URL: {sample[14]}")
        print("---------------\n")

# ページ数の自動検出関数
def get_total_pages(soup):
    try:
        page_links = soup.find('div', class_='pagination-parts').find_all('a')
        return int(page_links[-2].text)  # 最後から2番目が最大ページ数
    except:
        return 2000  # デフォルト値

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    try:
        html = requests.get(url, headers=headers, timeout=10)
        html.raise_for_status()
        soup = BeautifulSoup(html.content, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        logging.error(f"ページ読み込みエラー: {e}")
        raise

# CSV出力関数
def export_to_csv(conn):
    # データベースからデータを取得
    df = pd.read_sql_query("SELECT * FROM properties", conn)
    
    # CSV出力
    filename = f'suumo_properties_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\nCSVファイル {filename} に出力しました。")
    
    # データ概要を表示
    print("\n--- データ概要 ---")
    print(f"総物件数: {len(df)}")
    
    # 家賃の平均計算（数値に変換可能な場合のみ）
    try:
        df['rent_numeric'] = df['rent'].str.replace('万円', '').astype(float)
        print(f"平均家賃: {df['rent_numeric'].mean():.2f}万円")
    except:
        print("家賃の平均計算に失敗しました。")
    
    print(f"物件カテゴリ内訳:\n{df['category'].value_counts()}")

def main():
    # データベース初期化
    conn, cursor = init_database()
    
    # 複数ページの情報をまとめて取得
    data_samples = []

    # 最初のページで総ページ数を取得
    first_page_soup = load_page(url.format(1))
    max_page = get_total_pages(first_page_soup)
    
    # 処理時間を測りたい
    start = time.time()
    times = []

    # 均一な待機時間のための基準時間
    base_start_time = time.time()

    try:
        # ページごとの処理
        for page in range(1, max_page+1):
            # 均一な待機時間の実装
            ideal_elapsed_time = (page - 1) * 1.0  # ページごとに1秒間隔
            actual_elapsed_time = time.time() - base_start_time
            
            # 理想の経過時間より早く処理が進んでいる場合、待機
            if actual_elapsed_time < ideal_elapsed_time:
                time.sleep(ideal_elapsed_time - actual_elapsed_time)
            
            before = time.time()
            
            # ページ情報
            soup = load_page(url.format(page))
            
            # 物件情報リストを指定
            mother = soup.find_all(class_='cassetteitem')
            
            # 物件ごとの処理
            for child in mother:
                # 建物情報
                data_home = []
                
                # カテゴリ
                try:
                    data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
                except:
                    data_home.append('不明')
                
                # 建物名
                try:
                    data_home.append(child.find(class_='cassetteitem_content-title').text)
                except:
                    data_home.append('不明')
                
                # 住所
                try:
                    data_home.append(child.find(class_='cassetteitem_detail-col1').text)
                except:
                    data_home.append('不明')
                
                # 最寄り駅のアクセス
                try:
                    children = child.find(class_='cassetteitem_detail-col2')
                    stations = children.find_all(class_='cassetteitem_detail-text')
                    for id, grandchild in enumerate(stations):
                        data_home.append(grandchild.text)
                    # 2駅未満の場合、不足分を埋める
                    while len(data_home) < 6:
                        data_home.append('不明')
                except:
                    data_home.extend(['不明', '不明'])
                
                # 築年数と階数
                try:
                    children = child.find(class_='cassetteitem_detail-col3')
                    for grandchild in children.find_all('div'):
                        data_home.append(grandchild.text)
                except:
                    data_home.extend(['不明', '不明'])

                # 部屋情報
                rooms = child.find(class_='cassetteitem_other')
                for room in rooms.find_all(class_='js-cassette_link'):
                    data_room = []
                    
                    # 部屋情報が入っている表を探索
                    room_tds = room.find_all('td')
                    for id_, grandchild in enumerate(room_tds):
                        try:
                            # 階
                            if id_ == 2:
                                data_room.append(grandchild.text.strip())
                            
                            # 家賃と管理費
                            elif id_ == 3:
                                rent = grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text
                                management_fee = grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text
                                data_room.append(rent)
                                data_room.append(management_fee)
                            
                            # 敷金と礼金
                            elif id_ == 4:
                                deposit = grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text
                                gratuity = grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text
                                data_room.append(deposit)
                                data_room.append(gratuity)
                            
                            # 間取りと面積
                            elif id_ == 5:
                                layout = grandchild.find(class_='cassetteitem_madori').text
                                area = grandchild.find(class_='cassetteitem_menseki').text
                                data_room.append(layout)
                                data_room.append(area)
                            
                            # url
                            elif id_ == 8:
                                get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                                abs_url = urllib.parse.urljoin(url, get_url)
                                data_room.append(abs_url)
                        
                        except Exception as e:
                            data_room.append('不明')
                    
                    # 物件情報と部屋情報をくっつける
                    data_sample = data_home + data_room
                    data_samples.append(data_sample)
            
            # データベースに保存
            insert_to_database(conn, cursor, data_samples, page)
            
            # データサンプルをリセット
            data_samples = []
            
            # 進捗確認
            after = time.time()
            running_time = after - before
            times.append(running_time)
            
            print(f'{page}ページ目：{running_time}秒')
            
            # 作業進捗
            complete_ratio = round(page/max_page*100, 3)
            print(f'完了：{complete_ratio}%')
            
            # 作業の残り時間目安を表示
            running_mean = np.mean(times)
            running_required_time = running_mean * (max_page - page)
            hour = int(running_required_time/3600)
            minute = int((running_required_time%3600)/60)
            second = int(running_required_time%60)
            print(f'残り時間：{hour}時間{minute}分{second}秒\n')

            # ログ記録
            logging.info(f'ページ {page} スクレイピング完了')

    except Exception as e:
        logging.error(f"スクレイピング中にエラー発生: {e}")
        print(f"エラー: {e}")
    
    # 処理時間を測りたい
    finish = time.time()
    running_all = finish - start
    print('総経過時間：', running_all)

    # CSV出力
    export_to_csv(conn)

    # データベース接続を閉じる
    conn.close()
    logging.info('データベース接続終了')

if __name__ == "__main__":
    main()

: 